In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import locale
import json
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
import time
import statistics
import pandas_datareader.data as pdr

In [2]:
# Create a temporary sqlite database
database_connection_string = 'sqlite:///'

# Create an engine to interact with the database
engine = sqlalchemy.create_engine(database_connection_string)

In [3]:
today = date.today()

In [4]:
def get_data(tickers, start_date, end_date):
        
    # download daily price data for each of the stocks in the portfolio
    df = pdr.get_data_yahoo(stocks, start=start_date, end = end_date)['Adj Close']
    df.sort_index(inplace = True)
    
    return df

In [5]:
stocks = ['BTC-USD','ETH-USD','USDT-USD','BNB-USD','ADA-USD','XRP-USD','SOL-USD','DOGE-USD','DOT-USD','UNI-USD','USDC-USD','BUSD-USD','HEX-USD']

In [6]:
start_crypto_dates = ('06/01/2021') 
end_crypto_dates = (today)
crypto_prices = get_data(stocks,start_crypto_dates,end_crypto_dates)

#print(crypto_prices.head())
print(crypto_prices.tail())

Symbols          BTC-USD      ETH-USD  USDT-USD     BNB-USD   ADA-USD  \
Date                                                                    
2022-05-29  29445.957031  1812.031006  0.999178  305.975067  0.481126   
2022-05-30  31726.390625  1996.441284  0.999376  321.752899  0.570073   
2022-05-31  31792.310547  1942.328003  0.999388  320.485107  0.625254   
2022-06-01  29799.080078  1823.569336  0.999260  300.504059  0.552750   
2022-06-02  30523.326172  1835.599609  0.999368  307.960846  0.586519   

Symbols      XRP-USD    SOL-USD  DOGE-USD  DOT-USD   UNI-USD  USDC-USD  \
Date                                                                     
2022-05-29  0.389016  44.909615  0.082686      NaN  0.000147  1.000586   
2022-05-30  0.418175  47.179443  0.087871      NaN  0.000159  1.000099   
2022-05-31  0.421867  45.767841  0.085865      NaN  0.000159  1.000210   
2022-06-01  0.398085  40.269669  0.081056      NaN  0.000149  1.000130   
2022-06-02  0.405596  40.950073  0.082909  9

In [7]:
def get_historical_data(ticker, start_date):
    # Pull Historical Data
    data = yf.download(ticker, start=start_date)
    # Calculate Daily Returns
    data['Daily Return'] = data['Adj Close'].pct_change()   
    return data.dropna()

In [8]:
def std_dev(data):
    # Get number of observations
    n = len(data)
    # Calculate mean
    mean = sum(data) / n
    # Calculate deviations from the mean
    deviations = sum([(x - mean)**2 for x in data])
    # Calculate Variance & Standard Deviation
    variance = deviations / (n - 1)
    s = variance**(1/2)
    return s

In [9]:
# Sharpe Ratio From Scratch
def annualize_sharpe_ratio(data, risk_free_rate=0.0):
    # Calculate Average Daily Return
    mean_daily_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = std_dev(data)
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate) / s
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
    
    return sharpe_ratio

In [10]:
# Sharpe Ratio From Scratch
def daily_sharpe_ratio(data, risk_free_rate=0.0):
    # Calculate Average Daily Return
    mean_daily_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = std_dev(data)
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate) / s
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
    
    return daily_sharpe_ratio

In [11]:
display(stocks)
type (stocks)


['BTC-USD',
 'ETH-USD',
 'USDT-USD',
 'BNB-USD',
 'ADA-USD',
 'XRP-USD',
 'SOL-USD',
 'DOGE-USD',
 'DOT-USD',
 'UNI-USD',
 'USDC-USD',
 'BUSD-USD',
 'HEX-USD']

list

In [47]:
btc_sr = get_historical_data('BTC-USD',start_date='2021-01-01')
eth_sr = get_historical_data('ETH-USD',start_date='2022-01-01')
usdt_sr = get_historical_data('USDT-USD',start_date='2021-06-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [48]:
display (annualize_sharpe_ratio(btc_sr['Daily Return']))
annualize_sharpe_ratio(usdt_sr['Daily Return'])

0.3439014590710716

-0.06498152117616063

In [14]:
sharpe_ratio_BTC = daily_sharpe_ratio(btc_sr['Daily Return'])
sharpe_ratio_ETH = daily_sharpe_ratio(eth_sr['Daily Return'])
sharpe_ratio_USDT = daily_sharpe_ratio(usdt_sr['Daily Return'])
sharpe_ratio_USDT



-0.01946057028975582

In [15]:
# Next, we'll define a function for calculating daily returns for our coins
def calc_daily_returns(df):
    return (df.pct_change())

In [16]:
crypto_daily_rets = calc_daily_returns(crypto_prices)
print(crypto_daily_rets.tail())

Symbols      BTC-USD   ETH-USD  USDT-USD   BNB-USD   ADA-USD   XRP-USD  \
Date                                                                     
2022-05-29  0.021900  0.030769  0.000176 -0.005979  0.035205  0.006971   
2022-05-30  0.077445  0.101770  0.000198  0.051566  0.184873  0.074956   
2022-05-31  0.002078 -0.027105  0.000012 -0.003940  0.096796  0.008829   
2022-06-01 -0.062695 -0.061142 -0.000128 -0.062346 -0.115959 -0.056373   
2022-06-02  0.024304  0.006597  0.000108  0.024814  0.061093  0.018868   

Symbols      SOL-USD  DOGE-USD   DOT-USD   UNI-USD  USDC-USD  BUSD-USD  \
Date                                                                     
2022-05-29  0.012774  0.010350  0.000000  0.013793  0.000436 -0.000740   
2022-05-30  0.050542  0.062707  0.000000  0.081633 -0.000487  0.000484   
2022-05-31 -0.029920 -0.022829  0.000000  0.000000  0.000111 -0.001554   
2022-06-01 -0.120132 -0.056007  0.000000 -0.062893 -0.000080  0.000749   
2022-06-02  0.016896  0.022858 -0.127

In [17]:
# Calculating Sharpe Ratio Any Sharpe Ratio above 1.00 generally means the portfolio or stock is acceptable to hold because you are receiving an excess return per unit of risk.
Sharpe_Ratio = crypto_daily_rets.mean() / crypto_daily_rets.std()
display(Sharpe_Ratio)
type(Sharpe_Ratio)

Symbols
BTC-USD     0.004050
ETH-USD    -0.001163
USDT-USD   -0.003929
BNB-USD     0.011712
ADA-USD    -0.029218
XRP-USD    -0.024059
SOL-USD     0.044075
DOGE-USD   -0.042939
DOT-USD    -0.012360
UNI-USD     0.090640
USDC-USD    0.001023
BUSD-USD    0.003305
HEX-USD     0.046800
dtype: float64

pandas.core.series.Series

In [18]:
# converting to list
Crypto_list = Sharpe_Ratio.tolist()
Crypto_list

[0.004050029723450235,
 -0.001163074159829727,
 -0.0039290915809694045,
 0.011712084147079591,
 -0.02921826201686846,
 -0.02405903419376819,
 0.04407513534418283,
 -0.04293865830022385,
 -0.012359819675972545,
 0.09064001698150156,
 0.0010226950242389434,
 0.003304545218286936,
 0.046800393657985905]

In [19]:
df = pd.DataFrame (Crypto_list)
#df.columns = ['product_name', 'price','3','4','5','6','7','8','9','10','11','12','13']
print (df)

           0
0   0.004050
1  -0.001163
2  -0.003929
3   0.011712
4  -0.029218
5  -0.024059
6   0.044075
7  -0.042939
8  -0.012360
9   0.090640
10  0.001023
11  0.003305
12  0.046800


In [20]:
# Annualized Sharpe Ratio
A_Sharpe_Ratio = (252**0.5)*Sharpe_Ratio
display(A_Sharpe_Ratio)

Symbols
BTC-USD     0.064292
ETH-USD    -0.018463
USDT-USD   -0.062372
BNB-USD     0.185924
ADA-USD    -0.463826
XRP-USD    -0.381925
SOL-USD     0.699671
DOGE-USD   -0.681630
DOT-USD    -0.196206
UNI-USD     1.438866
USDC-USD    0.016235
BUSD-USD    0.052458
HEX-USD     0.742933
dtype: float64

In [21]:
# Define a function for calculating mean daily returns for our coins
def calc_mean_daily_returns(daily_returns):
    return (daily_returns.mean(skipna=True, level=None, numeric_only=None))

In [22]:
crypto_mean_daily_rets = calc_mean_daily_returns(crypto_daily_rets)
print (crypto_mean_daily_rets)

#round(crypto_mean)

Symbols
BTC-USD     1.459706e-04
ETH-USD    -5.015297e-05
USDT-USD   -1.906682e-06
BNB-USD     5.081989e-04
ADA-USD    -1.553254e-03
XRP-USD    -1.214409e-03
SOL-USD     2.854305e-03
DOGE-USD   -2.438451e-03
DOT-USD    -6.896913e-04
UNI-USD     3.604403e-02
USDC-USD    5.194577e-07
BUSD-USD    2.880736e-06
HEX-USD     3.778018e-03
dtype: float64


In [23]:
# Define a function for creating covariance matrices
def create_covariance_matrix(daily_returns):
    return daily_returns.cov()

In [24]:
crypto_cov_matrix = create_covariance_matrix(crypto_daily_rets)
print(crypto_cov_matrix)

Symbols        BTC-USD       ETH-USD      USDT-USD       BNB-USD   ADA-USD  \
Symbols                                                                      
BTC-USD   1.299016e-03  1.321454e-03 -3.025744e-07  1.212562e-03  0.001372   
ETH-USD   1.321454e-03  1.859424e-03  3.098494e-07  1.556069e-03  0.001708   
USDT-USD -3.025744e-07  3.098494e-07  2.354899e-07  1.403889e-06  0.000001   
BNB-USD   1.212562e-03  1.556069e-03  1.403889e-06  1.882778e-03  0.001705   
ADA-USD   1.371550e-03  1.708240e-03  1.086029e-06  1.704512e-03  0.002826   
XRP-USD   1.321729e-03  1.650229e-03  8.117063e-07  1.652544e-03  0.002036   
SOL-USD   1.406933e-03  1.946048e-03  2.344558e-06  1.832019e-03  0.002200   
DOGE-USD  1.383311e-03  1.722700e-03  2.357908e-06  1.765299e-03  0.001950   
DOT-USD   1.236528e-03  1.597652e-03 -1.053385e-06  1.598972e-03  0.001623   
UNI-USD   1.143378e-03  8.969902e-04 -3.608148e-06  1.703180e-03  0.003235   
USDC-USD -1.713122e-06 -2.312851e-06  1.069995e-07 -7.234160e-07

In [25]:
cs = crypto_prices
print(cs)
#statistics.variance(cs)

Symbols          BTC-USD      ETH-USD  USDT-USD     BNB-USD   ADA-USD  \
Date                                                                    
2021-06-01  36684.925781  2633.518311  1.000109  362.794647  1.734753   
2021-06-02  37575.179688  2706.125000  1.000550  401.262329  1.752824   
2021-06-03  39208.765625  2855.126465  1.000571  427.807922  1.840471   
2021-06-04  36894.406250  2688.195068  1.001038  391.395325  1.710908   
2021-06-05  35551.957031  2630.576904  1.000313  390.431854  1.659009   
...                  ...          ...       ...         ...       ...   
2022-05-29  29445.957031  1812.031006  0.999178  305.975067  0.481126   
2022-05-30  31726.390625  1996.441284  0.999376  321.752899  0.570073   
2022-05-31  31792.310547  1942.328003  0.999388  320.485107  0.625254   
2022-06-01  29799.080078  1823.569336  0.999260  300.504059  0.552750   
2022-06-02  30523.326172  1835.599609  0.999368  307.960846  0.586519   

Symbols      XRP-USD    SOL-USD  DOGE-USD    DOT-U

In [26]:
# The Free Crypto API Call endpoint URLs for the held cryptocurrency assets
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
usdt_url = "https://api.alternative.me/v2/ticker/Tether/?convert=USD"
bnb_url = "https://api.alternative.me/v2/ticker/BinanceCoin/?convert=USD"
ada_url = "https://api.alternative.me/v2/ticker/Cardano/?convert=USD"
xrp_url = "https://api.alternative.me/v2/ticker/Ripple/?convert=USD"
sol_url = "https://api.alternative.me/v2/ticker/Solana/?convert=USD"
doge_url = "https://api.alternative.me/v2/ticker/Dogecoin/?convert=USD"
dot_url = "https://api.alternative.me/v2/ticker/Polkadot/?convert=USD"
uni_url = "https://api.alternative.me/v2/ticker/Uniswap/?convert=USD"

In [27]:
btc_response = requests.get(btc_url).json()
eth_response= requests.get(eth_url).json()
usdt_response= requests.get(usdt_url).json()
bnb_response= requests.get(bnb_url).json()
ada_response = requests.get(ada_url).json()
xrp_response = requests.get(xrp_url).json()
sol_response = requests.get(sol_url).json()
doge_response = requests.get(doge_url).json()
dot_response = requests.get(dot_url).json()
uni_response= requests.get(uni_url).json()

In [28]:
#print(json.dumps(btc_response, indent=4, sort_keys=True))
#print(json.dumps(eth_response, indent=4, sort_keys=True))
#print(json.dumps(usdt_response, indent=4, sort_keys=True))
#print(json.dumps(bnb_response, indent=4, sort_keys=True))
#print(json.dumps(ada_response, indent=4, sort_keys=True))
#print(json.dumps(xrp_response, indent=4, sort_keys=True))
#print(json.dumps(sol_response, indent=4, sort_keys=True))
#print(json.dumps(doge_response, indent=4, sort_keys=True))
#print(json.dumps(dot_response, indent=4, sort_keys=True))
#print(json.dumps(uni_response, indent=4, sort_keys=True))

In [29]:
# Crypto names
btc_name = btc_response["data"]["1"]["name"]
eth_name = eth_response["data"]["1027"]["name"]
usdt_name = usdt_response["data"]["825"]["name"]
bnb_name = bnb_response["data"]["1839"]["name"]
ada_name = ada_response["data"]["2010"]["name"]
xrp_name = xrp_response["data"]["52"]["name"]
sol_name = sol_response["data"]["11733"]["name"]
doge_name = doge_response["data"]["74"]["name"]
dot_name = dot_response["data"]["11517"]["name"]
uni_name = uni_response["data"]["11968"]["name"]

In [30]:
# Crypto Symbol
btc_symbol = btc_response["data"]["1"]["symbol"]
eth_symbol = eth_response["data"]["1027"]["symbol"]
usdt_symbol = usdt_response["data"]["825"]["symbol"]
bnb_symbol = bnb_response["data"]["1839"]["symbol"]
ada_symbol = ada_response["data"]["2010"]["symbol"]
xrp_symbol = xrp_response["data"]["52"]["symbol"]
sol_symbol = sol_response["data"]["11733"]["symbol"]
doge_symbol = doge_response["data"]["74"]["symbol"]
dot_symbol = dot_response["data"]["11517"]["symbol"]
uni_symbol = uni_response["data"]["11968"]["symbol"]

In [31]:
# Crypto Category
btc_cat = 'BTC'
eth_cat = "Altcoins"
usdt_cat = "Stablecoin"
bnb_cat = "Stablecoin"
ada_cat = 'Altcoins'
xrp_cat = 'Altcoins'
sol_cat = 'Altcoins'
doge_cat = 'Meme coin'
dot_cat = 'Altcoins'
uni_cat = "Altcoins"

In [32]:
# Crypto price
btc_price = "${:,.2f}".format(btc_response["data"]["1"]["quotes"]["USD"]["price"])
eth_price = "${:,.2f}".format(eth_response["data"]["1027"]["quotes"]["USD"]["price"])
usdt_price = "${:,.2f}".format(usdt_response["data"]["825"]["quotes"]["USD"]["price"])
bnb_price = "${:,.2f}".format(bnb_response["data"]["1839"]["quotes"]["USD"]["price"])
ada_price = "${:,.2f}".format(ada_response["data"]["2010"]["quotes"]["USD"]["price"])
xrp_price = "${:,.2f}".format(xrp_response["data"]["52"]["quotes"]["USD"]["price"])
sol_price = "${:,.2f}".format(sol_response["data"]["11733"]["quotes"]["USD"]["price"])
doge_price = "${:,.2f}".format(doge_response["data"]["74"]["quotes"]["USD"]["price"])
dot_price = "${:,.2f}".format(dot_response["data"]["11517"]["quotes"]["USD"]["price"])
uni_price = "${:,.2f}".format(uni_response["data"]["11968"]["quotes"]["USD"]["price"])

In [33]:
print(btc_price)

$30,562.00


In [34]:
# 24hr Volume
btc_vol = f"$ ""{:,}".format(btc_response["data"]["1"]["quotes"]["USD"]["volume_24h"])
eth_vol = f"$ ""{:,}".format(eth_response["data"]["1027"]["quotes"]["USD"]["volume_24h"])
usdt_vol = f"$ ""{:,}".format(usdt_response["data"]["825"]["quotes"]["USD"]["volume_24h"])
bnb_vol = f"$ ""{:,}".format(bnb_response["data"]["1839"]["quotes"]["USD"]["volume_24h"])
ada_vol = f"$ ""{:,}".format(ada_response["data"]["2010"]["quotes"]["USD"]["volume_24h"])
xrp_vol = f"$ ""{:,}".format(xrp_response["data"]["52"]["quotes"]["USD"]["volume_24h"])
sol_vol = f"$ ""{:,}".format(sol_response["data"]["11733"]["quotes"]["USD"]["volume_24h"])
doge_vol = f"$ ""{:,}".format(doge_response["data"]["74"]["quotes"]["USD"]["volume_24h"])
dot_vol = f"$ ""{:,}".format(dot_response["data"]["11517"]["quotes"]["USD"]["volume_24h"])
uni_vol = f"$ ""{:,}".format(uni_response["data"]["11968"]["quotes"]["USD"]["volume_24h"])

In [35]:
# Market Cap
btc_cap = f"$ ""{:,}".format(btc_response["data"]["1"]["quotes"]["USD"]["market_cap"])
eth_cap = f"$ ""{:,}".format(eth_response["data"]["1027"]["quotes"]["USD"]["market_cap"])
usdt_cap = f"$ ""{:,}".format(usdt_response["data"]["825"]["quotes"]["USD"]["market_cap"])
bnb_cap = f"$ ""{:,}".format(bnb_response["data"]["1839"]["quotes"]["USD"]["market_cap"])
ada_cap = f"$ ""{:,}".format(ada_response["data"]["2010"]["quotes"]["USD"]["market_cap"])
xrp_cap = f"$ ""{:,}".format(xrp_response["data"]["52"]["quotes"]["USD"]["market_cap"])
sol_cap = f"$ ""{:,}".format(sol_response["data"]["11733"]["quotes"]["USD"]["market_cap"])
doge_cap =f"$ " "{:,}".format(doge_response["data"]["74"]["quotes"]["USD"]["market_cap"])
dot_cap = f"$ ""{:,}".format(dot_response["data"]["11517"]["quotes"]["USD"]["market_cap"])
uni_cap = f"$ ""{:,}".format(uni_response["data"]["11968"]["quotes"]["USD"]["market_cap"])

In [36]:
# percent_change_1h
btc_1hr = "{:.2%}".format((btc_response["data"]["1"]["quotes"]["USD"]["percentage_change_1h"] / 100))
eth_1hr = "{:.2%}".format((eth_response["data"]["1027"]["quotes"]["USD"]["percentage_change_1h"] / 100))
usdt_1hr = "{:.2%}".format((usdt_response["data"]["825"]["quotes"]["USD"]["percentage_change_1h"] / 100))
bnb_1hr = "{:.2%}".format((bnb_response["data"]["1839"]["quotes"]["USD"]["percentage_change_1h"] / 100))
ada_1hr = "{:.2%}".format((ada_response["data"]["2010"]["quotes"]["USD"]["percentage_change_1h"] / 100))
xrp_1hr = "{:.2%}".format((xrp_response["data"]["52"]["quotes"]["USD"]["percentage_change_1h"] / 100))
ada_1hr = "{:.2%}".format((ada_response["data"]["2010"]["quotes"]["USD"]["percentage_change_1h"] / 100))
sol_1hr = "{:.2%}".format((sol_response["data"]["11733"]["quotes"]["USD"]["percentage_change_1h"] / 100))
doge_1hr = "{:.2%}".format((doge_response["data"]["74"]["quotes"]["USD"]["percentage_change_1h"] / 100))
dot_1hr = "{:.2%}".format((dot_response["data"]["11517"]["quotes"]["USD"]["percentage_change_1h"] / 100))
uni_1hr = "{:.2%}".format((uni_response["data"]["11968"]["quotes"]["USD"]["percentage_change_1h"] / 100))

In [37]:
# percent_change_24h
btc_24hr = "{:.2%}".format((btc_response["data"]["1"]["quotes"]["USD"]["percentage_change_24h"] / 100))
eth_24hr = "{:.2%}".format((eth_response["data"]["1027"]["quotes"]["USD"]["percentage_change_24h"] / 100))
usdt_24hr = "{:.2%}".format((usdt_response["data"]["825"]["quotes"]["USD"]["percentage_change_24h"] / 100))
bnb_24hr = "{:.2%}".format((bnb_response["data"]["1839"]["quotes"]["USD"]["percentage_change_24h"] / 100))
ada_24hr = "{:.2%}".format((ada_response["data"]["2010"]["quotes"]["USD"]["percentage_change_24h"] / 100))
xrp_24hr = "{:.2%}".format((xrp_response["data"]["52"]["quotes"]["USD"]["percentage_change_24h"] / 100))
sol_24hr = "{:.2%}".format((sol_response["data"]["11733"]["quotes"]["USD"]["percentage_change_24h"] / 100))
doge_24hr= "{:.2%}".format((doge_response["data"]["74"]["quotes"]["USD"]["percentage_change_24h"] / 100))
dot_24hr = "{:.2%}".format((dot_response["data"]["11517"]["quotes"]["USD"]["percentage_change_24h"] / 100))
uni_24hr = "{:.2%}".format((uni_response["data"]["11968"]["quotes"]["USD"]["percentage_change_24h"] / 100))

In [38]:
# percent_change_7d
btc_7d =  "{:.2%}".format((btc_response["data"]["1"]["quotes"]["USD"]["percentage_change_7d"] / 100))
eth_7d =  "{:.2%}".format((eth_response["data"]["1027"]["quotes"]["USD"]["percentage_change_7d"] / 100))
usdt_7d =  "{:.2%}".format((usdt_response["data"]["825"]["quotes"]["USD"]["percentage_change_7d"] / 100))
bnb_7d =  "{:.2%}".format((bnb_response["data"]["1839"]["quotes"]["USD"]["percentage_change_7d"] / 100))
ada_7d = "{:.2%}".format((ada_response["data"]["2010"]["quotes"]["USD"]["percentage_change_7d"] / 100))
xrp_7d = "{:.2%}".format((xrp_response["data"]["52"]["quotes"]["USD"]["percentage_change_7d"] / 100))
sol_7d = "{:.2%}".format((sol_response["data"]["11733"]["quotes"]["USD"]["percentage_change_7d"] / 100))
doge_7d = "{:.2%}".format((doge_response["data"]["74"]["quotes"]["USD"]["percentage_change_7d"] / 100))
dot_7d = "{:.2%}".format((dot_response["data"]["11517"]["quotes"]["USD"]["percentage_change_7d"] / 100))
uni_7d =  "{:.2%}".format((uni_response["data"]["11968"]["quotes"]["USD"]["percentage_change_7d"] / 100))

In [39]:
stocks_dataframe = pd.DataFrame({'Coins': [btc_name,eth_name,usdt_name,bnb_name,ada_name,xrp_name,sol_name,doge_name,dot_name,uni_name], 'Symbol': [btc_symbol,eth_symbol,usdt_symbol,bnb_symbol,ada_symbol,xrp_symbol,sol_symbol,doge_symbol,dot_symbol,uni_symbol],'Category': [btc_cat,eth_cat,usdt_cat,bnb_cat,ada_cat,xrp_cat,sol_cat,doge_cat,dot_cat,uni_cat], 'Price': [btc_price,eth_price,usdt_price,bnb_price,ada_price,xrp_price,sol_price,doge_price,dot_price,uni_price],'24Hr Volume': [btc_vol,eth_vol,usdt_vol,bnb_vol,ada_vol,xrp_vol,sol_vol,doge_vol,dot_vol,uni_vol], 'Market Caplization':[btc_cap,eth_cap,usdt_cap,bnb_cap,ada_cap,xrp_cap,sol_cap,doge_cap,dot_cap,uni_cap],'1hr % Change': [btc_1hr,eth_1hr,usdt_1hr,bnb_1hr,ada_1hr,xrp_1hr,sol_1hr,dot_1hr,doge_1hr,uni_1hr], '24hrs %Change': [btc_24hr,eth_24hr,usdt_24hr,bnb_24hr,ada_24hr,xrp_24hr,sol_24hr,doge_24hr,dot_24hr,uni_24hr],'7days %Change': [btc_7d,eth_7d,usdt_7d,bnb_7d,ada_7d,xrp_7d,sol_7d,doge_7d,dot_7d,uni_7d], 'Sharpe Ratio': [sharpe_ratio_BTC,sharpe_ratio_ETH, sharpe_ratio_USDT,4,5,6,7,8,9,0],'Variance': [btc_name, 77, 3,4,5,6,7,8,9,0] })
stocks_dataframe.to_sql('Crypto', engine)
engine.table_names()

/var/folders/3v/j3665vvd1qn2w9n50h1qqmqr0000gn/T/ipykernel_14537/744152312.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Crypto']

In [40]:
#query = """
#SELECT "Coins" FROM Crypto;
#"""
#results = engine.execute(query)
#list(results)

In [41]:
stocks_dataframe

,Coins,Symbol,Category,Price,24Hr Volume,Market Caplization,1hr % Change,24hrs %Change,7days %Change,Sharpe Ratio,Variance
0,Bitcoin,BTC,BTC,"$30,562.00","$ 24,305,204,671","$ 583,209,879,136",0.03%,2.35%,3.30%,-0.066191,Bitcoin
1,Ethereum,ETH,Altcoins,"$1,838.33","$ 16,492,082,939","$ 222,824,298,949",-0.20%,0.39%,-5.48%,-0.092259,77
2,Tether,USDT,Stablecoin,$1.00,"$ 37,090,366,295","$ 72,579,497,534",0.05%,-0.00%,0.07%,-0.019461,3
3,Binance Coin,BNB,Stablecoin,$308.63,"$ 1,107,805,574","$ 50,467,551,902",0.24%,2.44%,-5.38%,4.000000,4
4,Cardano,ADA,Altcoins,$0.59,"$ 1,165,235,093","$ 19,891,700,712",0.19%,6.06%,14.29%,5.000000,5
5,XRP,XRP,Altcoins,$0.41,"$ 1,303,349,130","$ 19,665,438,440",0.26%,1.79%,0.09%,6.000000,6
6,Solana,SOL,Altcoins,$41.01,"$ 1,646,370,924","$ 13,982,426,446",-0.57%,1.62%,-14.67%,7.000000,7
7,Dogecoin,DOGE,Meme coin,$0.08,"$ 380,795,418","$ 11,025,408,618",-0.04%,2.67%,-0.50%,8.000000,8
8,Polkadot,DOT,Altcoins,$9.94,"$ 428,456,369","$ 11,129,926,426",0.16%,4.63%,0.87%,9.000000,9
9,Uniswap,UNI,Altcoins,$5.33,"$ 111,128,487","$ 2,433,091,025",0.59%,2.98%,-5.14%,0.000000,0
